In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

import networkx as nx
import sys

import tensorflow as tf
from tensorflow.math import scalar_mul as c_mul
from keras.callbacks import EarlyStopping

print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [2]:
OpenNEURO_dataset = loadmat("./Dataset/OpenNERO_32_and_132_data.mat")
OpenNEURO_dataset_132 = OpenNEURO_dataset['OpenNERO_132_roidata']
OpenNEURO_dataset_32 = OpenNEURO_dataset['OpenNERO_32_roidata']
OpenNEURO_labels = [0] * 117 + [1] * 49 # 0 - Control, 1 - Patient

X_train_data = OpenNEURO_dataset_32
y_train_data = OpenNEURO_labels

N, T, num_data = X_train_data.shape
print(N, T, num_data)

32 152 166


In [3]:
def generate_delta(T):
    diff_matrix = np.zeros([T, T - 1]) - np.eye(T, T - 1)
    for i in range(1, T):
        diff_matrix[i, i - 1] = 1
    return tf.convert_to_tensor(diff_matrix, dtype = "float32")

def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

def initial_graph(X_training, N, top_values = 6):
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    
    # top_values = 6
    sorted_indices = distance_matrix.argsort(1)
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
    
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    
    Laplacian_init = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return Laplacian_init

def label_print(idx, length):
    l = len(str(idx))
    if l < length:
        return "0" * (length - l) + str(idx)
    else:
        return str(idx)

In [4]:
d = generate_delta(T)
print(d)

Mul = np.eye(N) - np.ones([N, N])
Mul = tf.convert_to_tensor(Mul, dtype = "float32")

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1. -1.]
 [ 0.  0.  0. ...  0.  0.  1.]], shape=(152, 151), dtype=float32)


In [5]:
coeffs = 0
psi_cost = 0
Laplacian = 0
A_transform_cost = 0

model_pred = []
loss_model = []
learned_graph = []
mse_known_list = []
mse_unknown_list = []

In [6]:
coeffs_list = np.load("./Outputs/OpenNEURO_cost_func_coeffs.npy")
print(coeffs_list.shape)

(166, 4)


In [7]:
# A - Target, B - Predicted
def costfunc(A, B):
    global coeffs
    global X_train
    global psi_cost
    global Laplacian
    global A_transform_cost

    B_tmp = tf.squeeze(B)
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2) * coeffs[0]
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost) * coeffs[1]
    L_cost = (tf.norm(Laplacian) ** 2) * coeffs[2]
    Z_cost = (tf.norm(A_transform_cost) ** 2) * coeffs[3]
    
    tmp = (smoothness) + (data_fidelity) + (L_cost) + (Z_cost)
    
    global loss_model
    loss_model.append(tmp)
    
    return tmp
    
def costfunc1(A, B):
    global X_train
    global psi_cost
    
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2)
    return (data_fidelity)

def costfunc2(A, B):
    global Laplacian
    global A_transform_cost
    
    B_tmp = tf.squeeze(B)
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost)
    
    return (smoothness)

def costfunc3(A, B):
    global Laplacian
    
    L_cost = (tf.norm(Laplacian) ** 2)
    
    return (L_cost)

def costfunc4(A, B):
    global A_transform_cost
    
    Z_cost = (tf.norm(A_transform_cost) ** 2)
    
    return (Z_cost)

In [8]:
def mse_unknown(A, B):
    
    global psi_cost
    psi = psi_cost
    
    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B))) ** 2) / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):
    
    global psi_cost
    psi = psi_cost
    
    mse_kn = (LA.norm(np.multiply(psi, (X_train - B))) ** 2) / np.sum(psi.numpy().flatten())
    
    global model_pred
    model_pred.append([A.numpy(), B.numpy()])
    
    global mse_known_list
    mse_known_list.append(mse_kn)
    
    return mse_kn

In [9]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.0001, verbose = 1, patience = 10)

lr_decay_factor = 1.2
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [10]:
class ProposedModel(tf.keras.Model):
    
    def __init__(self, L_init, reg_init, loops_init, d_init, p_init, alpha_init, beta_init, threshold_in, **kwargs):
        super(ProposedModel, self).__init__(**kwargs)
        
        self.p1_init = tf.constant(p_init[0], shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        
        self.p2_init = tf.constant(p_init[1], shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        
        self.L_in = L_init
        self.preg = reg_init
        self.loops = loops_init
        self.d_int = d_init
        self.alp = alpha_init
        self.bet = beta_init
        self.threshold_init = threshold_in
        
    def data_inpainting(self, psi, Y, Lk, AAT):
        loop_b, reg = self.loops[1], self.preg

        Xk = tf.zeros_like(Y)
        Zk = -(tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT)))
        
        for i in range(loop_b):
            fdx_xk = tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT))

            # row_print = 4
            # tmp = list(zip((tf.multiply(psi, Xk) - Y)[:, row_print].numpy(), (2 * reg * (Lk @ Xk @ AAT)[:, row_print].numpy())))
            # print("\n\nStart here")
            # for k in tmp:
            #     print(k)
            
            fdx_zk = tf.multiply(psi, Zk) - Y + (2 * reg * (Lk @ Zk @ AAT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)

            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (2 * reg * (Lk @ Xk_1 @ AAT))
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)

            Zk_1 = (gamma * Zk) - fdx_xk_1

            Xk = Xk_1
            Zk = Zk_1
        # print("\n\nEnds here\n\n")
        return Xk_1 
    
    def graph_learning(self, X, Lk, AAT):
        loop_c, alpha, beta = self.loops[2], self.alp, self.bet
        
        # Computing the smoothness term
        X_AAT_XT = tf.matmul(X, tf.matmul(AAT, tf.transpose(X)))
        
        for i in range(loop_c):

            # Extracting the adjacency matrix from the Laplacian
            adj_i = tf.multiply(Mul, Lk)

            # Computing the gradient with respect to laplacian
            fdL_Lk = X_AAT_XT + (beta * Lk)
            
            # row_print = 4
            # print("For beta")
            # tmp = list(zip(X_AAT_XT[row_print, :].numpy(), np.array(beta * Lk)[row_print, :]))
            # for k in tmp:
            #     print(k)
            
            # Extracting the adjacency matrix from the gradient
            adj_grad = tf.multiply(Mul, fdL_Lk)
            
            # Computing the new adjacency
            # tmp = list(zip(adj_i[row_print, :].numpy(), (alpha * adj_grad)[row_print, :].numpy()))
            # if i == (loop_c-1):
            #     print("\nFor alpha")
            #     for k in tmp:
            #         print(k)
            #     print("\nStart here")
            
            adj_i1 = adj_i - (alpha * adj_grad)
            
            # Avoiding any negative values
            adj_i1 = tf.nn.relu(adj_i1)

            # Computing the new laplacian
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(adj_i1, 1)) - adj_i1
            
            # Updating the laplacian
            Lk = Lk_1
        # print("\n\nEnds here\n\n")
        return Lk
    

    def call(self, y):

        psi = y[0,:,T:]
        global psi_cost
        psi_cost = psi

        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0,:,:T]
        X_in = psi * tf.convert_to_tensor(X_in, dtype = "float32")
        Y = X_in
        
        d = self.d_int
        LT = tf.matmul(d, tf.transpose(d))
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        A_transform = tf.eye(T) + (self.p1 * MP(LT, 1)) + (self.p2 * MP(LT, 2))
        AAT = A_transform
        
        global A_transform_cost
        A_transform_cost = AAT
        
        loop_a = self.loops[0]
        threshold = self.threshold_init
        Lk = self.L_in
        
        for i in range(loop_a):
            
            Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
            
            Xk_1_gt = X_in + tf.multiply((tf.ones(psi.shape) - psi), Xk_1)
            
            Lk_1 = self.graph_learning(Xk_1_gt, Lk, AAT)
            
            Adj = tf.linalg.diag(tf.linalg.diag_part(Lk_1)) - Lk_1 # Gives adjacency
            
            n = tf.math.count_nonzero(Adj, dtype="float32")
            mean = tf.math.reduce_sum(Adj) / n
            Adj = tf.math.subtract(Adj, (threshold * mean)) # Ensures the sparsity of learned graph
            
            Adj = tf.nn.relu(Adj)
            # Adj = tf.math.sign(Adj)
            
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(Adj, 1)) - Adj
            
            Lk = Lk_1
        
            global Laplacian
            Laplacian = Lk
        
        Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
        
        global learned_graph
        learned_graph.append(Lk.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [11]:
loop_a = 5
loop_b = 15 # Data Inpainting
loop_c = 15 # Graph Learning

# reg_init = 2.0e-7
reg_init = 1.0e-7

# alpha_init = 1.0e-4
# beta_init = 1.0e1
alpha_init = 5.0e-4
beta_init = 1.0e1

threshold_init = 0.2

no_of_psi = 20
sensing_ratio = 0.1

p_init = [60.0, 120.0]
lr_decay_factor = 1.2

In [12]:
# all_vars = []
# for dat in range(num_data):
#     print(f"Datapoint: {dat + 1}")
#     model_pred = []
#     loss_model = []
#     learned_graph = []

#     X_train = X_train_data[:, :, dat]
#     Lap_init = initial_graph(X_train, N, 6)
    
#     all_psi = []

#     M = int(sensing_ratio * T)
#     X_train_missing = np.zeros([no_of_psi, N, T])
#     X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])

#     for i in range(no_of_psi):
#         psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
#         np.random.shuffle(psi_k)
#         psi_k = psi_k.reshape([N, T])
#         all_psi.append(psi_k)
#         X_train_missing[i, : , : ] = X_train * psi_k
#         X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
#     coeffs = coeffs_list[dat, :]

#     sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0001, momentum = 0.0)
    
#     our_model = ProposedModel(Lap_init, reg_init, [loop_a, loop_b, loop_c], 
#                               d, p_init, alpha_init, beta_init, threshold_init, name = 'our')
#     our_model.compile(optimizer = sgd_optimizer, loss = costfunc,
#                       metrics = [mse_known, mse_unknown], run_eagerly=True)
    
#     our_model.fit(X_train_concatenated, X_train_missing, epochs = 15, 
#                   callbacks = [lr_scheduler], batch_size = 1)
    
#     var = our_model.variables
#     var_lst = []
#     for v in var:
#         var_lst.append(v.numpy()[0, 0])
#     print(f"\nFilter Coefficients:", var_lst)
#     all_vars.append(var_lst)

#     graph = learned_graph[-1]
#     print(f"\nLearned Graph Shape:\n{graph.shape}")

#     np.save(f"./Outputs/Graphs/OpenNEURO/{label_print(dat, 3)}_{y_train_data[dat]}.npy", graph)

In [13]:
# np.save("./Outputs/OpenNEURO_mse_known.npy", mse_known_list)
# np.save("./Outputs/OpenNEURO_mse_unknown.npy", mse_unknown_list)

In [14]:
# re_run_idx = [12, 37, 81, 97, 118, 124, 126, 127, 129, 133, 134, 135, 142, 143, 146, 151]

# all_vars = []
# for dat in re_run_idx:
#     dat = dat - 1
#     print(f"Datapoint: {dat + 1}")
#     model_pred = []
#     loss_model = []
#     learned_graph = []

#     X_train = X_train_data[:, :, dat]
#     Lap_init = initial_graph(X_train, N, 6)
    
#     all_psi = []

#     M = int(sensing_ratio * T)
#     X_train_missing = np.zeros([no_of_psi, N, T])
#     X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])

#     for i in range(no_of_psi):
#         psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
#         np.random.shuffle(psi_k)
#         psi_k = psi_k.reshape([N, T])
#         all_psi.append(psi_k)
#         X_train_missing[i, : , : ] = X_train * psi_k
#         X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
#     coeffs = coeffs_list[dat, :]

#     sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 0.00001, momentum = 0.0)
    
#     our_model = ProposedModel(Lap_init, reg_init, [loop_a, loop_b, loop_c], 
#                               d, p_init, alpha_init, beta_init, threshold_init, name = 'our')
#     our_model.compile(optimizer = sgd_optimizer, loss = costfunc,
#                       metrics = [mse_known, mse_unknown], run_eagerly=True)
    
#     our_model.fit(X_train_concatenated, X_train_missing, epochs = 5, 
#                   callbacks = [lr_scheduler], batch_size = 1)
    
#     var = our_model.variables
#     var_lst = []
#     for v in var:
#         var_lst.append(v.numpy()[0, 0])
#     print(f"\nFilter Coefficients:", var_lst)
#     all_vars.append(var_lst)

#     graph = learned_graph[-1]
#     print(f"\nLearned Graph Shape:\n{graph.shape}")

#     np.save(f"./Outputs/Graphs/OpenNEURO/{label_print(dat, 3)}_{y_train_data[dat]}.npy", graph)

Datapoint: 12
Epoch 1/5
20/20 [==============================] - 19s 953ms/step - loss: 3720360.5000 - mse_known: 0.6825 - mse_unknown: 1.7570 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 18s 904ms/step - loss: 3707797.5000 - mse_known: 0.6853 - mse_unknown: 1.7562 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 20s 1s/step - loss: 3695926.7500 - mse_known: 0.6877 - mse_unknown: 1.7558 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 18s 900ms/step - loss: 3685540.5000 - mse_known: 0.6898 - mse_unknown: 1.7553 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 12s 578ms/step - loss: 3677168.5000 - mse_known: 0.6916 - mse_unknown: 1.7550 - lr: 6.9444e-06



Filter Coefficients: [56.510418, 126.00794]

Learned Graph Shape:
(32, 32)
Datapoint: 37
Epoch 1/5
20/20 [==============================] - 12s 582ms/step - loss: 1583636.0000 - mse_known: 0.1919 - mse_unknown: 0.7688 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 17s 850ms/step - loss: 1581566.7500 - mse_known: 0.1924 - mse_unknown: 0.7683 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 17s 883ms/step - loss: 1579506.3750 - mse_known: 0.1929 - mse_unknown: 0.7680 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 16s 815ms/step - loss: 1577613.3750 - mse_known: 0.1934 - mse_unknown: 0.7677 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 17s 833ms/step - loss: 1576039.8750 - mse_known: 0.1938 - mse_unknown: 0.7674 - lr: 6.9444e-06



Filter Coefficients: [59.004414, 122.72794]

Learned Graph Shape:
(32, 32)
Datapoint: 81
Epoch 1/5
20/20 [==============================] - 15s 767ms/step - loss: 831436.8750 - mse_known: 0.0694 - mse_unknown: 0.4808 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 17s 828ms/step - loss: 831280.8125 - mse_known: 0.0694 - mse_unknown: 0.4809 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 16s 815ms/step - loss: 831124.3125 - mse_known: 0.0694 - mse_unknown: 0.4810 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 16s 780ms/step - loss: 830979.6250 - mse_known: 0.0693 - mse_unknown: 0.4811 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 16s 799ms/step - loss: 830860.4375 - mse_known: 0.0693 - mse_unknown: 0.4812 - lr: 6.9444e-06



Filter Coefficients: [59.20032, 119.95752]

Learned Graph Shape:
(32, 32)
Datapoint: 97
Epoch 1/5
20/20 [==============================] - 16s 784ms/step - loss: 821717.4375 - mse_known: 0.0665 - mse_unknown: 0.4476 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 27s 1s/step - loss: 821507.5000 - mse_known: 0.0664 - mse_unknown: 0.4478 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 28s 1s/step - loss: 821295.0000 - mse_known: 0.0664 - mse_unknown: 0.4479 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 26s 1s/step - loss: 821100.1875 - mse_known: 0.0663 - mse_unknown: 0.4480 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 26s 1s/step - loss: 820930.8125 - mse_known: 0.0663 - mse_unknown: 0.4481 - lr: 6.9444e-06



Filter Coefficients: [59.06666, 119.97349]

Learned Graph Shape:
(32, 32)
Datapoint: 118
Epoch 1/5
20/20 [==============================] - 24s 1s/step - loss: 5096435.0000 - mse_known: 1.7462 - mse_unknown: 3.2485 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 30s 1s/step - loss: 4982652.0000 - mse_known: 1.7340 - mse_unknown: 3.2528 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 21s 1s/step - loss: 4889397.0000 - mse_known: 1.7223 - mse_unknown: 3.2576 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 17s 832ms/step - loss: 4818905.5000 - mse_known: 1.7120 - mse_unknown: 3.2617 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 19s 959ms/step - loss: 4770989.0000 - mse_known: 1.7037 - mse_unknown: 3.2654 - lr: 6.9444e-06


...


Filter Coefficients: [57.640263, 117.156334]

Learned Graph Shape:
(32, 32)
Datapoint: 135
Epoch 1/5
20/20 [==============================] - 15s 774ms/step - loss: 728825.1875 - mse_known: 0.0382 - mse_unknown: 0.3352 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 17s 847ms/step - loss: 727392.1875 - mse_known: 0.0380 - mse_unknown: 0.3354 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 17s 877ms/step - loss: 725922.3750 - mse_known: 0.0378 - mse_unknown: 0.3357 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 21s 1s/step - loss: 724572.3750 - mse_known: 0.0376 - mse_unknown: 0.3360 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 18s 911ms/step - loss: 723443.8125 - mse_known: 0.0375 - mse_unknown: 0.3362 - lr: 6.9444e-06



Filter Coefficients: [58.68508, 117.955154]

Learned Graph Shape:
(32, 32)
Datapoint: 142
Epoch 1/5
20/20 [==============================] - 18s 914ms/step - loss: 1000714.6250 - mse_known: 0.0685 - mse_unknown: 0.3936 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 20s 1s/step - loss: 1000276.0000 - mse_known: 0.0684 - mse_unknown: 0.3938 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 19s 929ms/step - loss: 999836.1250 - mse_known: 0.0682 - mse_unknown: 0.3939 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 19s 969ms/step - loss: 999413.1875 - mse_known: 0.0681 - mse_unknown: 0.3940 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 15s 774ms/step - loss: 999066.1875 - mse_known: 0.0680 - mse_unknown: 0.3941 - lr: 6.9444e-06



Filter Coefficients: [58.878155, 119.26126]

Learned Graph Shape:
(32, 32)
Datapoint: 143
Epoch 1/5
20/20 [==============================] - 19s 919ms/step - loss: 1103284.1250 - mse_known: 0.0675 - mse_unknown: 0.4466 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 16s 808ms/step - loss: 1102182.8750 - mse_known: 0.0672 - mse_unknown: 0.4468 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 19s 929ms/step - loss: 1101063.2500 - mse_known: 0.0669 - mse_unknown: 0.4470 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 16s 794ms/step - loss: 1099997.6250 - mse_known: 0.0667 - mse_unknown: 0.4471 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 20s 985ms/step - loss: 1099103.7500 - mse_known: 0.0664 - mse_unknown: 0.4473 - lr: 6.9444e-06



Filter Coefficients: [58.848053, 118.185455]

Learned Graph Shape:
(32, 32)
Datapoint: 146
Epoch 1/5
20/20 [==============================] - 20s 1s/step - loss: 767285.3750 - mse_known: 0.0710 - mse_unknown: 0.3610 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 20s 992ms/step - loss: 766615.2500 - mse_known: 0.0708 - mse_unknown: 0.3612 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 15s 755ms/step - loss: 765883.0000 - mse_known: 0.0707 - mse_unknown: 0.3615 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 14s 702ms/step - loss: 765270.3750 - mse_known: 0.0706 - mse_unknown: 0.3617 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 14s 716ms/step - loss: 764736.6250 - mse_known: 0.0705 - mse_unknown: 0.3619 - lr: 6.9444e-06



Filter Coefficients: [58.587578, 119.189]

Learned Graph Shape:
(32, 32)
Datapoint: 151
Epoch 1/5
20/20 [==============================] - 15s 733ms/step - loss: 1206623.7500 - mse_known: 0.2902 - mse_unknown: 0.7398 - lr: 1.0000e-05
Epoch 2/5
20/20 [==============================] - 14s 715ms/step - loss: 1205140.7500 - mse_known: 0.2907 - mse_unknown: 0.7396 - lr: 1.0000e-05
Epoch 3/5
20/20 [==============================] - 18s 921ms/step - loss: 1203727.2500 - mse_known: 0.2912 - mse_unknown: 0.7395 - lr: 1.0000e-05
Epoch 4/5
20/20 [==============================] - 18s 889ms/step - loss: 1202483.5000 - mse_known: 0.2916 - mse_unknown: 0.7394 - lr: 8.3333e-06
Epoch 5/5
20/20 [==============================] - 17s 867ms/step - loss: 1201485.7500 - mse_known: 0.2919 - mse_unknown: 0.7393 - lr: 6.9444e-06

Filter Coefficients: [59.702103, 122.374374]

Learned Graph Shape:
(32, 32)
